In [ ]:
# Import credentials
from resource_credentials import (subscription_key, 
    resource_group_name, 
    workspace_name,
    datastore_name,
    storage_account_name,
    blob_container_name,
    storage_account_key,
    data_asset_name)

from azure.ai.ml import MLClient
from azure.ai.ml.entities import Workspace
from azure.identity import DefaultAzureCredential

## Create Workspace

In [ ]:
# Create a ml client
ml_client = MLClient(credential=DefaultAzureCredential(), 
                     subscription_id=subscription_key, 
                     resource_group_name=resource_group_name)

# Define the workspace
ws = Workspace(name=workspace_name, location="eastus") 

ws = ml_client.workspaces.begin_create(ws).result()
print(ws)

### List workspaces

In [ ]:
current_workspaces = ml_client.workspaces.list()
for workspace in current_workspaces:
    print(workspace.name)

## Create and register a Datastore

In [ ]:
from azure.ai.ml.entities import AzureBlobDatastore, AccountKeyConfiguration

ml_client = MLClient(credential=DefaultAzureCredential(), 
                     subscription_id=subscription_key, 
                     resource_group_name=resource_group_name,
                     workspace_name=workspace_name)
# Define the datastore
datastore = AzureBlobDatastore(name=datastore_name, 
                                account_name=storage_account_name, 
                                container_name=blob_container_name, 
                                credentials=AccountKeyConfiguration(account_key = storage_account_key ))

datastore = ml_client.create_or_update(datastore)

### Add a dataset to a datastore

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

# Get the datastore
datastore = ml_client.datastores.get(datastore_name)

# Define path with the datastore name
path = "azureml://datastores/" + datastore_name + "/paths/loan-data/Loan+Approval+Prediction.csv"
data_asset = Data(name=data_asset_name, 
                  type=AssetTypes.URI_FILE, 
                  path=path)

ml_client.data.create_or_update(data_asset)